In [ ]:

from config.database import client, collection_weather
from config.database import collection_flights, client_UJ, db_UJ        # UJ mongoDB
from core.root_class import Root_class, Fetching_Mechanism, Source_links_and_api
from bson import ObjectId
import requests
import datetime as dt
import json
import pickle
from pymongo import UpdateOne

# db = client_UJ.cirrostrats                 # get a particular db
cts = db_UJ['test_st']   # create/get a collection
collection = collection_flights
collection_st = collection_searchTrack
# collection_backup = client_UJ['cirrostrats']['flights_backup']
# collection = collection_searchTrack


In [2]:
collection_flights.find_one()

{'_id': ObjectId('67e4c89528d60c5468f28aec'),
 'flightID': 'DAL927',
 'versions': [{'_id': ObjectId('68fcb2632f36c7d9de21b02a'),
   'timestamp': datetime.datetime(2025, 10, 25, 11, 16, 1, 451000),
   'aircraftType': 'A21N',
   'registration': 'N595DT',
   'departure': 'KATL',
   'arrival': 'KLAS',
   'estimatedDepartureTime': '2025-10-25T12:05:00Z',
   'route': 'KATL.CUTTN2.HANKO..HLI..OKM..MMB..KA39Y..KA36U..GUP..HAHAA.RKSTR4.KLAS/0340',
   'requestedAltitude': '30000.0',
   'version_created_at': datetime.datetime(2025, 10, 25, 11, 20, 3, 211000)},
  {'_id': ObjectId('68fcc00a2f36c7d9de21c195'),
   'timestamp': datetime.datetime(2025, 10, 25, 12, 17, 35, 116000),
   'organization': 'DAL',
   'aircraftType': 'A21N',
   'registration': 'N595DT',
   'departure': 'KATL',
   'arrival': 'KLAS',
   'route': 'KATL.CUTTN2.HANKO..HLI..OKM..MMB..KA39Y..KA36U..GUP..HAHAA.RKSTR4.KLAS/1557',
   'assignedAltitude': '30000.0',
   'currentBeacon': '7111',
   'version_created_at': datetime.datetime(202

In [7]:

search_index_collection = db_UJ['search_index']


In [9]:
search_index_collection.find_one({})


{'_id': ObjectId('6821b9805795b7ff557e3180'),
 'r_id': ObjectId('66176711170d1d57a24dfdb0'),
 'airport_st': 'RIC - Richmond International Airport',
 'ph': 0.7131475206485398,
 'submits': [datetime.datetime(2025, 8, 6, 21, 32, 38, 72000),
  datetime.datetime(2025, 8, 10, 1, 12, 0, 963000),
  datetime.datetime(2025, 9, 26, 18, 17, 15, 760000),
  datetime.datetime(2025, 9, 28, 22, 5, 54, 230000),
  datetime.datetime(2025, 9, 29, 13, 10, 42, 909000),
  datetime.datetime(2025, 9, 29, 13, 10, 56, 405000),
  datetime.datetime(2025, 9, 29, 18, 13, 55, 899000),
  datetime.datetime(2025, 9, 30, 3, 12, 54, 155000),
  datetime.datetime(2025, 10, 11, 23, 55, 25, 764000),
  datetime.datetime(2025, 10, 14, 12, 55, 7, 582000)]}

In [ ]:
# Inspect document hierarchy - type of field uniques
    # Helpful to understand schema variations by field type across documents
pipeline = [
    {"$project": {"fields": {"$objectToArray": "$$ROOT"}}},
    {"$unwind": "$fields"},
    {"$group": {"_id": "$fields.k", "types": {"$addToSet": {"$type": "$fields.v"}}}},
    {"$project": {"field": "$_id", "_id": 0, "types": 1}}
]

for doc in collection.aggregate(pipeline):
    print(doc)

{'types': ['objectId'], 'field': '_id'}
{'types': ['string'], 'field': 'flightID'}
{'types': ['array'], 'field': 'versions'}


In [ ]:
# Inspect hierarchy of fields - usage and presence of fields across documents
pipeline = [
        {
            "$project": {
                "arrayOfKeyValue": {"$objectToArray": "$$ROOT"}
            }
        },
        {"$unwind": "$arrayOfKeyValue"},
        {
            "$group": {
                "_id": "$arrayOfKeyValue.k",
                "count": {"$sum": 1},
                "percentage": {
                    "$avg": {
                        "$cond": [{"$ifNull": ["$arrayOfKeyValue.v", False]}, 1, 0]
                    }
                }
            }
        },
        {
            "$project": {
                "field": "$_id",
                "count": 1,
                "percentage": {"$multiply": ["$percentage", 100]},
                "_id": 0
            }
        },
        {"$sort": {"count": -1}}
    ]
    
list(search_index_collection.aggregate(pipeline))


[{'count': 3787, 'field': '_id', 'percentage': 100.0},
 {'count': 3787, 'field': 'ph', 'percentage': 100.0},
 {'count': 3736, 'field': 'r_id', 'percentage': 100.0},
 {'count': 3639, 'field': 'fid_st', 'percentage': 100.0},
 {'count': 298, 'field': 'submits', 'percentage': 100.0},
 {'count': 97, 'field': 'airport_st', 'percentage': 100.0},
 {'count': 51, 'field': 'Terminal/Gate', 'percentage': 100.0}]

In [5]:
collection_flights.find_one()

{'_id': ObjectId('67e4c89528d60c5468f28aec'),
 'flightID': 'DAL927',
 'versions': [{'_id': ObjectId('68fcb2632f36c7d9de21b02a'),
   'timestamp': datetime.datetime(2025, 10, 25, 11, 16, 1, 451000),
   'aircraftType': 'A21N',
   'registration': 'N595DT',
   'departure': 'KATL',
   'arrival': 'KLAS',
   'estimatedDepartureTime': '2025-10-25T12:05:00Z',
   'route': 'KATL.CUTTN2.HANKO..HLI..OKM..MMB..KA39Y..KA36U..GUP..HAHAA.RKSTR4.KLAS/0340',
   'requestedAltitude': '30000.0',
   'version_created_at': datetime.datetime(2025, 10, 25, 11, 20, 3, 211000)},
  {'_id': ObjectId('68fcc00a2f36c7d9de21c195'),
   'timestamp': datetime.datetime(2025, 10, 25, 12, 17, 35, 116000),
   'organization': 'DAL',
   'aircraftType': 'A21N',
   'registration': 'N595DT',
   'departure': 'KATL',
   'arrival': 'KLAS',
   'route': 'KATL.CUTTN2.HANKO..HLI..OKM..MMB..KA39Y..KA36U..GUP..HAHAA.RKSTR4.KLAS/1557',
   'assignedAltitude': '30000.0',
   'currentBeacon': '7111',
   'version_created_at': datetime.datetime(202

In [ ]:
# Inspect hierarchy - Find unique document Structures - Helpful when you have multiple variations of document in a collection
collection = collection_flights


def find_different_structures(collection, sample_size=5):
    """Find documents with different field structures"""
    pipeline = [
        {
            "$limit": 1000
        },
        {
            "$group": {
                "_id": {
                    "fields": {
                        "$map": {
                            "input": {"$objectToArray": "$$ROOT"},
                            "as": "field",
                            "in": "$$field.k"
                        }
                    }
                },
                "count": {"$sum": 1},
                "samples": {"$push": {"_id": "$_id", "doc": "$$ROOT"}}
            }
        },
        {
            "$project": {
                "fields": "$_id.fields",
                "count": 1,
                "sample_docs": {"$slice": ["$samples", sample_size]},
                "_id": 0
            }
        },
        {"$sort": {"count": -1}},
        {"$limit": 10}
    ]
    
    return list(collection.aggregate(pipeline, allowDiskUse=True))

# Usage
structures = find_different_structures(collection)
for i, struct in enumerate(structures):
    print(f"\nStructure {i+1}: {struct['count']} documents")
    print(f"Fields: {sorted(struct['fields'])}")
    print("Sample _id:", struct['sample_docs'][0]['_id'])

# learn.mongodb.com      -- Validation of the schema
# use validation to inspect schema for invalid documents - thru inverse validation 
# give me docs that dont adhere to my docs
# After that sequential update many operations to fix those invalid docs
#  Like if you find such docs x



# Atlas search - b uild index

# create text based search
# single collection pattern




Structure 1: 772 documents
Fields: ['_id', 'flightID', 'versions']
Sample _id: 67e4c89528d60c5468f28aec

Structure 2: 228 documents
Fields: ['_id', 'flightID']
Sample _id: 67e4c89528d60c5468f28af0


In [3]:
collection_flights.count_documents({})

193300

In [2]:
count_crit = {'ph':{"$exists":True}}       # return ones with popularity hits
# self.return_crit = {'ph':0}       # return only flightID and ph
c_sti_docs = list(cts.find(count_crit).sort('ph',-1))     # Reverse sort


In [ ]:

return_criteria = {'flightID':1,'versions':1}
a = collection_flights.find({'flightID': {'$regex': 'N787MG'}}, return_criteria)  # find a document with flightID containing x.

# collection_flights.count_documents({})  # count all documents in the collection
# list(a)

In [34]:
    
pipeline = [
    {"$limit": 10000},
    # Unwind the versions array to examine each version individually
    # {"$unwind": "$versions"},
    
    # Filter to keep only versions that have a clearance field
    {"$match": 
        # {"versions.clearance": {"$exists": True}}},
    
    {'flightID': {'$regex': "^UCA"}}},
    # Project the specific fields you need
    {"$project": {'flightID':1, '_id':0}}
        # "c": "$versions"  # Rename the versions object to clearance_data for clarity
    # }},
    
    # Optional: If you only want one result per flight, you can group by flightID
    # {"$group": {
    #     "_id": "$flightID",
    #     "clearance_data": {"$first": "$clearance_data"}
    # }}
]
# Execute the pipeline on your collection
# clearance_documents = list(collection_flights.aggregate(pipeline))


In [ ]:
# # Create a test collection. Takes about a few 10 seconds. 
# collection_name = 'test_airports'
# db = client.cirrostrats                 # get a particular db
# collection_test = db[collection_name]   # create/get a collection
# docs = [doc for doc in collection_airports_cache.find({}).limit(1000)]
# collection_test.insert_many(docs)
# cta = collection_test

# # collection_es = collection_test
# # id = [i['flightID'] for i in collection_es.find({},{'flightID':1, '_id':0})]


In [ ]:
# Mongo 101 - guide

# list all db
# client.list_database_names()
# db = client['cirrostrats']            # create a db
# collection = db['name_of_collection'] # create/get a collection. if it doesn't exist then get one.

# db = client.cirrostrats               # get a particular db
# all_db = db.list_collection_names()   # get all collections of the db
# collection = db['flights']            # get a particular collection


# ________________________________________________________

# get all documents. Expensive operation.
# x = [i for i in collection.find({})]

# return the first document
# one = collection.find_one({})

# Count documents
# collection_backup.count_documents({})

# Delete all documents from the collection
# collection_backup.delete_many({})

# return the first document that have a `count` field using the $exists operator
# collection.find_one({'count':{"$exists":True}})
# collection.find({'submits': { '$exists': 'false' } }))              # The other way around

# returns just the flightID and excludes ObjectID fields. Very fast and efficient. first argument is the filter by and second is the projection. In this case, 
# filters all documents(first argument {}) and projection(second argument) returns just the flightID field and excludes the ObjectID.
# criteria = {'flightID': 1, '_id': 0}
# collection.find({}, criteria)

# Get backup flight numbers from collection_backup(inefficient since it scans complete documents instead of just the flightNumber field) and insert them into the the empty collection
# Avoid use since its inefficient.
# docs = [{'flightNumber':i['flightNumber']} for i in collection_backup.find({})]
# collection.insert_many(docs)

# skim throguh only the flightID field from the collection(efficient) documents instead of getting all the fields from document.
# docs = [{'flightID':doc['flightID']} 
        # for doc in collection_other.find({}, {'flightID': 1, '_id': 0})]
# collection.insert_many(docs)

# add flight numbers to the collection- fast operation. does not account for uniques and wont avoid duplicates.
# docs = [{'flightNumber':i} for i in flightNumbers]
# collection.insert_many(docs)

# insert a key field and value in a document using set operator:
# collection.update_one({'flightID':'GJS123'},{'$set':{'count':7.92193}})               # first arg is find_crit second arg is set_crit

# rename a field within collection
# collection.update_many(
#     {'airport_st': {"$exists": True}},                        # find_crit
#     {'$rename': {'airport_st': 'airportDisplayTerm'}}         # perform_crit
# )

# unset operator - remove a field from a document
# collection.update_one({'flightID':'GJS123'},{'$unset':{'count':''}})


# This will insert flightnumbers in bulk in the collection if it doesnt exist - slow operation since
# it checks if the flightnumber already exist. !! WHY NOT SKIM THROUGH JUST flightID field instead of complete documents?
# update_operations = []
# docs = list(collection.find({}, {'_id': 0}))  # Exclude `_id` to prevent conflicts
# for i in docs:
#     flightNumber = i['flightID']
#     update_operations.append(
#         UpdateOne({'flightID': flightNumber},
#                   {'$set': {'flightID': flightNumber}},
#                   upsert=True
#                   )
#     ),
# result = collection_backup.bulk_write(update_operations)

# match all documents. works like find. will fetch all that matches gjs in `flightNumber` fields
# [i for i in collection_flights.find({'flightNumber':{'$regex': 'GJS'}})]

# match from the list provided.
# list_of_items_to_be_matched = ['x','y','z',]  # for exmple this  could a list of flight numbers you want to match within the collection.
# cfid = list(collection_flights.find({'flightID': {'$in': list(list_of_items_to_be_matched)}},{'flightID':1,} ))

# limit documents to first 300. Saves memory and intensive operations.
# a = collection.find().limit(300)

# Copy entire collection to a new collection named 'test_new'
# og_collection.aggregate([{'$out': 'test_new'}])

In [ ]:
# Advanced mongo operations - pipepline and aggregation.

# Typical use..
# pipeline = [
    # {"$match": {"count": {"$exists": True}}},        # filter documents that have a count field
    # {'$project': {'flightNumber': 1, '_id': 0}},
    # {'$out': 'collection_flights'}
# ]


# # collection_flights.find_one({})
# # MongoDB aggregation pipeline to find all documents with a 'clearance' field in any nested dictionary within the 'versions' array
# pipeline = [
#     {"$limit": 10000},
#     # Unwind the versions array to examine each version individually
#     {"$unwind": "$versions"},
    
#     # Filter to keep only versions that have a clearance field
#     {"$match": {"versions.clearance": {"$exists": True}}},
    
#     # Project the specific fields you need
#     {"$project": {
#         "c": "$versions"  # Rename the versions object to clearance_data for clarity
#     }},
    
#     # Optional: If you only want one result per flight, you can group by flightID
#     # {"$group": {
#     #     "_id": "$flightID",
#     #     "clearance_data": {"$first": "$clearance_data"}
#     # }}
# ]
# # Execute the pipeline on your collection
# clearance_documents = list(collection_flights.aggregate(pipeline))


# filter match documents according to the query and sort. the $or operator matches either `flightNumber` or `name`(for airport collection)
# collection_merge = [collection_airports, collection_flights] 
# for coll in collection_merge:
#     pipeline = [
#             {"$match": {"count": {"$exists": True}}},        # filter documents that have a count field
#             {"$match": {"$or": [
#                 {"flightNumber": {"$regex": query, "$options": "i"}},       # matches flightNumber field in flights collection
#                 {"name": {"$regex": query, "$options": "i"}}                # matches name field in airport collection
#             ]}},
#             {"$sort": {"count": -1}},               # sort by popularity - the count field contains popularity rating.
#         ]
#     coll.aggregate(pipeline)
#     suggestions.extend(coll.aggregate(pipeline))


# Only return flightID from the root doc and version_created_at fields from the versions subdocument.
#     '$project': {
#         'flightID': 1,
#         'version_timestamps': '$versions.version_created_at'

# Change name field to uppercase in collection, e.g 'name': 'New York', will change to 'name': 'NEW YORK'
    # '$project': {
    #   'name': { '$toUpper': "$name" },

# Group documents by field and sum their unique counts. e.g the 'route' field will be analysed for uniqueness and their counts.
    # $group: {
    #   _id: "$route",
    #   count: { $sum: 1 }



# collection.aggregate(pipeline)



In [ ]:
# Returning all airports in the US with their ICAO and IATA codes -- checking consistency between the two codes. e.g KEWR and EWR vs KUNV and SCE
icao = db_UJ['icao_iata']
# icao.count_documents({'country_code': 'US'})
find_criteria = {'country_code': 'US'}
# us_airports = icao.find(find_criteria,{'icao': 1, 'iata': 1, 'airport':1,'_id': 0})
us_airports = icao.find(find_criteria,{'icao': 1, 'iata': 1,'_id': 0})
us_airports = list(us_airports)

for i in us_airports:
    two = list(i.values())
    if type(two[1])!=float and str(two[1]) != 'nan':
        if two[1][1:] == two[0]:
            pass
            # print(two[0], two[1])
        else:
            print(two[0], two[1])
        # print(str(two[1])== 'nan')
    # if two[1] != float('nan'):
        # print(two[0], two[1])


In [21]:
# redis call
import redis
r = redis.Redis(host="localhost", port=6379, db=0, decode_responses=True)


In [ ]:
# redis call 2
# import json

# all_keys = [i for i in r.keys("*")]
# all_keys.remove('multipleEntries')

# db = [json.loads(r.get(i)) for i in a]

In [ ]:
# Part 1: forMDB 
import requests
xx = requests.get("http://3.15.228.76:8000/flights")

repo_flights = []
scheduled_flights = []
for i in xx.json()['flightNumbers']:
    # first three digits are GJS, after those digits are flight number
    # if len(i) == 7 and i[:3] == 'UAL' and i[3:].isnumeric():
    if i[:3] == 'UAL' and i[3:].isnumeric():
        # if i[3] == '4':
        scheduled_flights.append(int(i[3:]))
        # elif i[3] == '3':
            # repo_flights.append(int(i[3:]))

jmsFlightNumbers = xx.json()['flightNumbers']
# Saving these list of flight numbers into the collection flightNumbers. !! WONT ACCOUNT FOR DUPLICATES!
# collection_flightNumbers.insert_many(jmsFlightNumbers)

import pickle
with open('JMS_flight_numbers.pkl', 'wb') as f:
    pickle.dump(jmsFlightNumbers, f)

from routes.root.weather_parse import Weather_parse
from routes.root.dep_des import Pull_flight_info

tots = {}
flt_info = Pull_flight_info()
for i in scheduled_flights:
    rets = flt_info.fs_dep_arr_timezone_pull(i)
    tots[rets['origin_fs']] = tots.get(rets['origin_fs'],0) + 1
    tots[rets['destination_fs']] = tots.get(rets['destination_fs'],0) + 1
    
# Sorting the dictionary by values
tots = {k: v for k, v in sorted(tots.items(), key=lambda item: item[1], reverse=True)}

new_tots = {}
for icao,flightIDs in tots.items():
    sum_of_values = sum(tots.values())
    new_tots[icao] = int(flightIDs/sum_of_values*100)+1

forMDB = {'scheduled_flights': scheduled_flights, 'repo_flights': repo_flights, 'popularity_raw': tots, 'popularity_proportions': new_tots}

with open(r'forMDB.pkl', 'wb') as f:
    pickle.dump(forMDB, f)


In [ ]:
# Part 2 forMDB
# adds `count` as 1 to all gjs flights 
with open(r'forMDB.pkl', 'rb') as f:
    forMDB = pickle.load(f)
GJS_scheduled_flights = forMDB['scheduled_flights']
GJS_repo_flights = forMDB['repo_flights']
popularity_raw = forMDB['popularity_raw']
popularity_proportions = forMDB['popularity_proportions']

# 
operations = []
for i in GJS_scheduled_flights:
# for airport, count in popularity_proportions.items():
    flightNumber = "GJS"+str(i)
    operation = UpdateOne(
        {'flightNumber': flightNumber},
        {'$set': {'count': 1}},
        upsert=True
    )
    operations.append(operation)

# Perform bulk write
result = collection_flights.bulk_write(operations)
# popularity_proportions
# collection_flights.find_one()

In [86]:
gjs_reg = {'N503GJ',
 'N504GJ',
 'N506GJ',
 'N508GJ',
 'N511GJ',
 'N520GJ',
 'N521GJ',
 'N522GJ',
 'N523GJ',
 'N524GJ',
 'N526GJ',
 'N534GJ',
 'N535GJ',
 'N536GJ',
 'N537GJ',
 'N538GJ',
 'N539GJ',
 'N540GJ',
 'N541GJ',
 'N543GJ',
 'N544GJ',
 'N546GJ',
 'N548GJ',
 'N549GJ',
 'N551GJ',
 'N552GJ',
 'N554GJ',
 'N556GJ',
 'N557GJ',
 'N559GJ',
 'N561GJ',
 'N563GJ',
 'N564GJ',
 'N566GJ',
 'N569GJ',
 'N578GJ',
 'N579GJ',}